In [ ]:
from torchHHT import hht
import torch, os
import pandas as pd

: 

In [ ]:
def imfs_FEMTO(fs, directory:str, bearing:str):

    for fileName in os.listdir(directory):
        filePath = os.path.join(directory, fileName)

        if(filePath[22:26] == 'temp'): #skip all temperature data
            continue

        col_names = ['Hour', 'Minute', 'Second', 'Nano_second', 'Horizontal', 'Vertical']

        # femto1_7 dataset is seperated by comma others by semicolon
        if (bearing == "Bearing1_7"):
            data = pd.read_csv(filePath, sep=',', names=col_names) 
        else:
            data = pd.read_csv(filePath, sep=';', names=col_names)
        
        horizontal = data['Horizontal']
        vertical = data['Vertical']

        tensor_horizontal = torch.Tensor(horizontal.values)
        tensor_vertical = torch.Tensor(vertical.values)

        tensor_combined = torch.sqrt(torch.pow(tensor_horizontal, 2) + torch.pow(tensor_vertical, 2))


        if torch.cuda.is_available():
            device = torch.device("cuda:0")
            tensor_horizontal = tensor_horizontal.cuda()
            tensor_vertical = tensor_vertical.cuda()
            tensor_combined = tensor_combined.cuda()

        imfs_horizontal, imfs_env_horizontal, imfs_freq_horizontal = hht.hilbert_huang(tensor_horizontal, fs, num_imf=3)
        imfs_vertical, imfs_env_vertical, imfs_freq_vertical = hht.hilbert_huang(tensor_vertical, fs, num_imf=3)
        imfs_combined, imfs_env_combined, imfs_freq_combined = hht.hilbert_huang(tensor_combined, fs, num_imf=3)

        imf_string_horizontal = "imfs_" + bearing + "_horizontal_" + fileName[:-4]  + ".pt"
        imf_env_string_horizontal = "imfs_env_" + bearing + "_horizontal_" + fileName[:-4]  + ".pt"
        imf_freq_string_horizontal = "imfs_freq_" + bearing + "_horizontal_" + fileName[:-4]  + ".pt"

        imf_string_vertical = "imfs_" + bearing + "_vertical_" + fileName[:-4]  + ".pt"
        imf_env_string_vertical = "imfs_env_" + bearing + "_vertical_" + fileName[:-4]  + ".pt"
        imf_freq_string_vertical = "imfs_freq_" + bearing + "_vertical_" + fileName[:-4]  + ".pt"

        imf_string_combined = "imfs_" + bearing + "_combined_" + fileName[:-4]  + ".pt"
        imf_env_string_combined = "imfs_env_" + bearing + "_combined_" + fileName[:-4]  + ".pt"
        imf_freq_string_combined = "imfs_freq_" + bearing + "_combined_" + fileName[:-4]  + ".pt"

        folder_IMF_hor = "IMFs/FEMTO/" + bearing + "/horizontal"
        folder_IMF_ver = "IMFs/FEMTO/" + bearing + "/vertical"
        folder_IMF_com = "IMFs/FEMTO/" + bearing + "/combined"

        folder_IMF_ENV_hor = "IMF_ENVs/FEMTO/" + bearing + "/horizontal"
        folder_IMF_ENV_ver = "IMF_ENVs/FEMTO/" + bearing + "/vertical"
        folder_IMF_ENV_com = "IMF_ENVs/FEMTO/" + bearing + "/combined"

        folder_IMF_FREQ_hor = "IMF_FREQs/FEMTO/" + bearing + "/horizontal"
        folder_IMF_FREQ_ver = "IMF_FREQs/FEMTO/" + bearing + "/vertical"
        folder_IMF_FREQ_com = "IMF_FREQs/FEMTO/" + bearing + "/combined"
        
        torch.save(imfs_horizontal, os.path.join(folder_IMF_hor, imf_string_horizontal))
        torch.save(imfs_vertical, os.path.join(folder_IMF_ver, imf_string_vertical))
        torch.save(imfs_combined, os.path.join(folder_IMF_com, imf_string_combined))

        torch.save(imfs_env_horizontal, os.path.join(folder_IMF_ENV_hor, imf_env_string_horizontal))
        torch.save(imfs_env_vertical, os.path.join(folder_IMF_ENV_ver, imf_env_string_vertical))
        torch.save(imfs_env_combined, os.path.join(folder_IMF_ENV_com, imf_env_string_combined))
        
        torch.save(imfs_freq_horizontal, os.path.join(folder_IMF_FREQ_hor, imf_freq_string_horizontal))
        torch.save(imfs_freq_vertical, os.path.join(folder_IMF_FREQ_ver, imf_freq_string_vertical))
        torch.save(imfs_freq_combined, os.path.join(folder_IMF_FREQ_com, imf_freq_string_combined))

: 

In [ ]:
%%time
fs = 25600
imfs_FEMTO(fs, directory = "data/FEMTO/Bearing1_3", bearing = "Bearing1_3")
imfs_FEMTO(fs, directory = "data/FEMTO/Bearing1_4", bearing = "Bearing1_4")
imfs_FEMTO(fs, directory = "data/FEMTO/Bearing1_7", bearing = "Bearing1_7")

: 